In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#PREPARE TRAINING SENTENCES
! pip install ekphrasis
! pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 7.7 MB/s eta 0:00:00


In [ ]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'ekphrasis'

### Read Datasets

In [ ]:
# Specify the path to your CSV file
csv_file = '/content/drive/Shareddrives/HLT/dataset/development/training_textual.csv'
dev_data = pd.read_csv(csv_file)
dev_data.head()

In [ ]:
csv_file = '/content/drive/Shareddrives/HLT/dataset/gold/test_textual_gold.csv'
test_textual_gold = pd.read_csv(csv_file)
test_textual_gold.head()

,anonymized_tweet_id,anonymized_text,label,dataset
0,424801448454884,questo anno #babbonatale portato giocattolo it...,1,test_politics
1,775194088981616,Adesso che in mezzo alla strada grazie al verg...,1,test_politics
2,386928936347190,Circondatevi di persone che non sono diventate...,0,test_politics
3,957546674360002,"Seriamente, per capire, se un ladro mi entra i...",0,test_politics
4,530526299555950,Che poi è probabile che #Spataro non sia potut...,0,test_politics


### Data prerpocessing pipeline

In [ ]:
! python /usr/local/lib/python3.10/dist-packages/ekphrasis/tools/generate_stats.py --input /content/drive/Shareddrives/HLT/dataset/italian_corpus.txt --name tweet_ita --ngrams 2 --mincount 70 30


computing statistics for file:  /content/drive/Shareddrives/HLT/dataset/italian_corpus.txt
100% 10013615/10013615 [07:02<00:00, 23692.75it/s]

Writing 1-grams...
entries:326,541	-	tokens:174,118,463
writing stats to file /usr/local/lib/python3.10/dist-packages/ekphrasis/tools/../stats/tweet_ita/counts_1grams.txt
Writing 2-grams...
entries:5,653,502	-	tokens:174,118,463
writing stats to file /usr/local/lib/python3.10/dist-packages/ekphrasis/tools/../stats/tweet_ita/counts_2grams.txt


In [ ]:
text_processor = TextPreProcessor (
    # terms that will be normalized
    normalize=['url', 'email', 'user', 'percent', 'money', 'phone', 'time', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag"} ,
    fix_html=True ,  # fix HTML tokens
    #all_caps_tag="wrap", # how to wrap the capitalized words
    unpack_hashtags=True ,  # perform word segmentation on hashtags
    fix_text=True, #choose if you want to fix bad unicode terms and html entities
    segmenter="/usr/local/lib/python3.10/dist-packages/ekphrasis/stats/tweet_ita", # define the statistics of what corpus you would like to use
    corrector="/usr/local/lib/python3.10/dist-packages/ekphrasis/stats/tweet_ita", # same as before
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize
)

/usr/local/lib/python3.10/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading /usr/local/lib/python3.10/dist-packages/ekphrasis/stats/tweet_ita - 1grams ...
generating cache file for faster loading...
reading ngrams /usr/local/lib/python3.10/dist-packages/ekphrasis/stats/tweet_ita/counts_1grams.txt
Reading /usr/local/lib/python3.10/dist-packages/ekphrasis/stats/tweet_ita - 2grams ...
generating cache file for faster loading...
reading ngrams /usr/local/lib/python3.10/dist-packages/ekphrasis/stats/tweet_ita/counts_2grams.txt
Reading /usr/local/lib/python3.10/dist-packages/ekphrasis/stats/tweet_ita - 1grams ...


/usr/local/lib/python3.10/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [ ]:
def clean_disguised_bad_words(text):
    text = " " + text + " "
    #print(text)
    text = re.sub(r' vaff[xx]+lo ', ' vaffanculo ', text) # no need?
    text = re.sub(r' vaffan[xx**]+lo ', ' vaffanculo ', text) # no need?
    text = re.sub(r' aff[xx]+lo ', ' affanculo ', text)
    text = re.sub(r' affan[xx**]+lo ', ' affanculo ', text)
    text = re.sub(r' c[glo.x*@%#$^]+i ', ' coglioni ', text)
    text = re.sub(r' c[glo.x*@%#$^]+e ', ' coglione ', text)
    text = re.sub(r' ca\*\*o ', ' cazzo ', text, flags=re.IGNORECASE)
    text = re.sub(r' c[az.x*@%#$^]+ro ', ' cazzaro ', text)
    text = re.sub(r' inc[az.x*@%#$^]+to ', ' incazzato ', text)
    text = re.sub(r' c[az.x*@%#$^]+te ', ' cazzate ', text)
    text = re.sub(r' c[az.x*@%#$^]+i ', ' cazzi ', text)
    text = re.sub(r' c[az.x*@%#$^]+o ', ' cazzo ', text)
    text = re.sub(r' cazxxx ', ' cazzate ', text)
    text = re.sub(r' CAZO ', ' CAZZO ', text)
    text = re.sub(r' c[az.x*@%#$^]+one ', ' cazzone ', text)
    text = re.sub(r' m\*\*\*a ', ' merda ', text, flags=re.IGNORECASE)
    text = re.sub(r' m[erd.x*@%#$^]+@ ', ' merda ', text)
    text = re.sub(r' m[erd.x*@%#$^]+e ', ' merde ', text)
    text = re.sub(r' c[.x*@%#$^]+lo ', ' culo ', text)
    text = re.sub(r' culx ', ' culo ', text)
    text = re.sub(r' p[u.x*@%#$^]+ana ', ' puttana ', text)
    text = re.sub(r' p[tu.x*@%#$^]+e ', ' puttane ', text)
    text = re.sub(r' t[.x*@%#$^]+a ', ' troia ', text)
    text = re.sub(r' t[.x*@%#$^]+e ', ' troie ', text)
    text = re.sub(r' s[.x*@%#$^]+o ', ' stronzo ', text)
    text = re.sub(r' stronz[.x*@%#$^]+a ', ' stronzata ', text)
    text = re.sub(r' s[.x*@%#$^]+i ', ' stronzi ', text)
    text = re.sub(r' min[*t]+a ', ' minchiata ', text)
    return text

In [ ]:
from itertools import groupby

def deleteDuplicate (riga):
    parole = riga.split()
    stringa = ""
    for a in parole:
        parola = a
        a = [list(g) for k, g in groupby(a)]
        vocali = ['a','e','i','o','u','y']

        for idx,_ in enumerate(a):
            if idx == 0:
                stringa += a[idx][0]
            elif idx == len(a)-1:
                stringa += a[idx][0]
            elif a[idx][0] in vocali and (parola not in ['alcool', 'preesame']):
                stringa += a[idx][0]
            elif len(a[idx]) == 1:
                stringa += a[idx][0]
            elif len (a[idx]) >= 2:
                stringa += a[idx][0]
                stringa += a[idx][1]
        stringa =  stringa + " "
    return(stringa)

In [ ]:
import re
import emoji

def alberto_preprocessing(sentences):

  examples = []

  for s in sentences:
      #s = s.lower() #lowercasing
      s = re.sub(r'\[URL\]', '<url>', s) # URL Replacement:
      s = re.sub(r'#[Mm]5[Ss]', '#movimentocinquestelle', s)
      s = re.sub(r'[Mm]5[Ss]', 'movimentocinquestelle', s)
      #s = emoji.demojize(s, language='it') # Converts emojis into a textual representation
      s = clean_disguised_bad_words(s) # clean up disguised bad words
      s = deleteDuplicate(s)
      s = str(" ".join(text_processor.pre_process_doc(s)))
      s = re.sub(r"[^a-zA-ZÀ-ú</>!?\d\s]", ' ', s) # Removes characters that are not letters,digits, HTML-like tags,punctuation marks (! and ?), spaces, or Unicode characters in the range \U00010000-\U0010ffff.
      s = re.sub(r'<([^>]+)>', r'[\1]', s) #transforms tag from <> </> to [] [/]
      s = re.sub(r'\d+', '[number]', s)
      s = re.sub(r"\s+", ' ', s) #  Replaces multiple consecutive whitespace characters with a single space
      # s = re.sub(r'(\w)\1{2,}',r'\1\1', s) Reduce sequences where the same character is repeated three or more times to just two repetitions (e.g., "hellooooo" to "helloo").
      s = re.sub ( r'^\s' , '' , s ) # removes leading whitespace characters
      s = re.sub ( r'\s$' , '' , s ) # removes trailing whitespace characters
      #print(s)
      examples.append(s)
  return examples

### DEVELOPMENT

In [ ]:
sentences = dev_data.iloc[:,1]

In [ ]:
dev_data['anonymized_text'] = alberto_preprocessing(sentences)
dev_data

,anonymized_tweet_id,anonymized_text,label,dataset
0,217874450618134,[user] con tutte le denunce che si sta beccand...,1,training_politics
1,360042217507605,[hashtag] prescrizione [/hashtag] i t re magi ...,0,training_politics
2,817917484817935,il [hashtag] movimento cinque stelle [/hashtag...,1,training_politics
3,172580609652325,la [hashtag] lega [/hashtag] e il [hashtag] mo...,0,training_politics
4,145836038456701,che cosa cambia questa [hashtag] legge [/hasht...,0,training_politics
...,...,...,...,...
5595,715670659156960,[hashtag] covid [/hashtag] [hashtag] draghi [/...,0,training_politics
5596,512309888296620,cina [number] [number] [number] mila euro a ne...,0,training_politics
5597,887061502405620,[user] [user] [user] anch io leggo i commenti ...,1,training_politics
5598,531900858879345,trasporto pubblico simeone nominato consiglier...,0,training_politics


In [ ]:
from pathlib import Path
filepath = Path('/content/drive/Shareddrives/HLT/dataset/development/umberto_preprocessig/umberto_preprocessing_training_textual.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
dev_data.to_csv(filepath,index=False)

### GOLD TEST


In [ ]:
test_sentences = test_textual_gold.iloc[:,1]

In [ ]:
test_textual_gold['anonymized_text'] = alberto_preprocessing(test_sentences)
test_textual_gold

,anonymized_tweet_id,anonymized_text,label,dataset
0,424801448454884,questo anno [hashtag] babbo natale [/hashtag] ...,1,test_politics
1,775194088981616,adesso che in mezzo alla strada grazie al verg...,1,test_politics
2,386928936347190,circondatevi di persone che non sono diventate...,0,test_politics
3,957546674360002,seriamente per capire se un ladro mi entra in ...,0,test_politics
4,530526299555950,che poi è probabile che [hashtag] spataro [/ha...,0,test_politics
...,...,...,...,...
4395,391599381944152,[user] esatto brava ti dico di piu i nazisti n...,0,test_religious
4396,974317501584882,[user] togliere la parola non vaccinati e mett...,0,test_religious
4397,739414582677163,[user] [user] lo sai o no che i musulmani rico...,0,test_religious
4398,470933865054049,berlino memoriale per gli ebrei assassinati d ...,0,test_religious


In [ ]:
filepath = Path('/content/drive/Shareddrives/HLT/dataset/gold/umberto_preprocessing/umberto_preprocessing_test_textual.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
test_textual_gold.to_csv(filepath,index=False)

### Merged training


In [ ]:
merged_training = pd.read_csv('/content/drive/Shareddrives/HLT/dataset/development/merged_training.csv')
merged_training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4883 entries, 0 to 4882
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   anonymized_tweet_id     4883 non-null   int64  
 1   anonymized_text         4883 non-null   object 
 2   label                   4883 non-null   int64  
 3   dataset                 4883 non-null   object 
 4   retweet_count           4883 non-null   float64
 5   favorite_count          4883 non-null   float64
 6   anonymized_user_id      4883 non-null   float64
 7   statuses_count          4883 non-null   float64
 8   followers_count         4883 non-null   float64
 9   friends_count           4883 non-null   float64
 10  anonymized_description  4883 non-null   object 
dtypes: float64(6), int64(2), object(3)
memory usage: 419.8+ KB


In [ ]:
merged_training['anonymized_text'] = alberto_preprocessing(merged_training['anonymized_text'])
merged_training['anonymized_description'] = alberto_preprocessing(merged_training['anonymized_description'])

In [ ]:
merged_training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4883 entries, 0 to 4882
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   anonymized_tweet_id     4883 non-null   int64  
 1   anonymized_text         4883 non-null   object 
 2   label                   4883 non-null   int64  
 3   dataset                 4883 non-null   object 
 4   retweet_count           4883 non-null   float64
 5   favorite_count          4883 non-null   float64
 6   anonymized_user_id      4883 non-null   float64
 7   statuses_count          4883 non-null   float64
 8   followers_count         4883 non-null   float64
 9   friends_count           4883 non-null   float64
 10  anonymized_description  4883 non-null   object 
dtypes: float64(6), int64(2), object(3)
memory usage: 419.8+ KB


In [ ]:
merged_training.to_csv('/content/drive/Shareddrives/HLT/dataset/development/umberto_preprocessig/merged_umberto_preprocessing_training.csv', index=False)

### Merged test

In [ ]:
merged_test = pd.read_csv('/content/drive/Shareddrives/HLT/dataset/gold/merged_test.csv')
merged_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3740 entries, 0 to 3739
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   anonymized_tweet_id     3740 non-null   int64 
 1   anonymized_text         3740 non-null   object
 2   label                   3740 non-null   int64 
 3   dataset                 3740 non-null   object
 4   retweet_count           3740 non-null   int64 
 5   favorite_count          3740 non-null   int64 
 6   anonymized_user_id      3740 non-null   int64 
 7   statuses_count          3740 non-null   int64 
 8   followers_count         3740 non-null   int64 
 9   friends_count           3740 non-null   int64 
 10  anonymized_description  3740 non-null   object
dtypes: int64(8), object(3)
memory usage: 321.5+ KB


In [ ]:
merged_test['anonymized_text'] = alberto_preprocessing(merged_test['anonymized_text'])
merged_test['anonymized_description'] = alberto_preprocessing(merged_test['anonymized_description'])

In [ ]:
merged_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3740 entries, 0 to 3739
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   anonymized_tweet_id     3740 non-null   int64 
 1   anonymized_text         3740 non-null   object
 2   label                   3740 non-null   int64 
 3   dataset                 3740 non-null   object
 4   retweet_count           3740 non-null   int64 
 5   favorite_count          3740 non-null   int64 
 6   anonymized_user_id      3740 non-null   int64 
 7   statuses_count          3740 non-null   int64 
 8   followers_count         3740 non-null   int64 
 9   friends_count           3740 non-null   int64 
 10  anonymized_description  3740 non-null   object
dtypes: int64(8), object(3)
memory usage: 321.5+ KB


In [ ]:
merged_test.to_csv('/content/drive/Shareddrives/HLT/dataset/gold/umberto_preprocessing/merged_umberto_preprocessing_test.csv', index=False)

## Bug Fixing

In [ ]:
merged_training_proc = pd.read_csv('/content/drive/Shareddrives/HLT/dataset/development/umberto_preprocessig/merged_umberto_preprocessing_training.csv')

In [ ]:
merged_training_proc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4883 entries, 0 to 4882
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   anonymized_tweet_id     4883 non-null   int64  
 1   anonymized_text         4883 non-null   object 
 2   label                   4883 non-null   int64  
 3   dataset                 4883 non-null   object 
 4   retweet_count           4883 non-null   float64
 5   favorite_count          4883 non-null   float64
 6   anonymized_user_id      4883 non-null   float64
 7   statuses_count          4883 non-null   float64
 8   followers_count         4883 non-null   float64
 9   friends_count           4883 non-null   float64
 10  anonymized_description  4861 non-null   object 
dtypes: float64(6), int64(2), object(3)
memory usage: 419.8+ KB


In [ ]:
merged_training[merged_training_proc["anonymized_description"].notna()].shape

(4861, 11)

In [ ]:
merged_training_no_null_after_proc=merged_training[merged_training_proc["anonymized_description"].notna()].copy()

In [ ]:
merged_training_no_null_after_proc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4861 entries, 0 to 4882
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   anonymized_tweet_id     4861 non-null   int64  
 1   anonymized_text         4861 non-null   object 
 2   label                   4861 non-null   int64  
 3   dataset                 4861 non-null   object 
 4   retweet_count           4861 non-null   float64
 5   favorite_count          4861 non-null   float64
 6   anonymized_user_id      4861 non-null   float64
 7   statuses_count          4861 non-null   float64
 8   followers_count         4861 non-null   float64
 9   friends_count           4861 non-null   float64
 10  anonymized_description  4861 non-null   object 
dtypes: float64(6), int64(2), object(3)
memory usage: 455.7+ KB


In [ ]:
merged_training_no_null_after_proc.to_csv('/content/drive/Shareddrives/HLT/dataset/development/umberto_preprocessig/merged_umberto_preprocessing_training.csv', index=False)

In [ ]:
merged_training_proc = pd.read_csv('/content/drive/Shareddrives/HLT/dataset/development/umberto_preprocessig/merged_umberto_preprocessing_training.csv')

In [ ]:
merged_training_proc.head()

,anonymized_tweet_id,anonymized_text,label,dataset,retweet_count,favorite_count,anonymized_user_id,statuses_count,followers_count,friends_count,anonymized_description
0,217874450618134,[user] con tutte le denunce che si sta beccand...,1,training_politics,0.0,6.0,8.868893e+14,554.0,748.0,753.0,avete presente quegli stereotipi sui siciliani...
1,360042217507605,[hashtag] prescrizione [/hashtag] i t re magi ...,0,training_politics,154.0,448.0,2.570741e+14,35043.0,10838.0,4535.0,prof di latino e deputata pd ama lo sport la r...
2,817917484817935,il [hashtag] movimento cinque stelle [/hashtag...,1,training_politics,195.0,638.0,6.908473e+14,139750.0,7971.0,1866.0,medico appassionato di cinema d autore musica ...
3,145836038456701,che cosa cambia questa [hashtag] legge [/hasht...,0,training_politics,45.0,143.0,5.308388e+14,11234.0,10815.0,1810.0,veneto ! assessore regionale sviluppo economic...
4,740684329660693,in pochi giorni il [hashtag] ministro [/hashta...,0,training_politics,2.0,8.0,7.081260e+14,473.0,1171.0,1353.0,napoletano tifoso della fiorentina e rivoluzio...


In [ ]:
merged_test_proc = pd.read_csv('/content/drive/Shareddrives/HLT/dataset/gold/umberto_preprocessing/merged_umberto_preprocessing_test.csv')

In [ ]:
merged_test_proc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3740 entries, 0 to 3739
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   anonymized_tweet_id     3740 non-null   int64 
 1   anonymized_text         3740 non-null   object
 2   label                   3740 non-null   int64 
 3   dataset                 3740 non-null   object
 4   retweet_count           3740 non-null   int64 
 5   favorite_count          3740 non-null   int64 
 6   anonymized_user_id      3740 non-null   int64 
 7   statuses_count          3740 non-null   int64 
 8   followers_count         3740 non-null   int64 
 9   friends_count           3740 non-null   int64 
 10  anonymized_description  3703 non-null   object
dtypes: int64(8), object(3)
memory usage: 321.5+ KB


In [ ]:
merged_test[merged_test_proc["anonymized_description"].notna()].shape

(3703, 11)

In [ ]:
merged_test_no_null_after_proc=merged_test[merged_test_proc["anonymized_description"].notna()].copy()

In [ ]:
merged_test_no_null_after_proc.to_csv('/content/drive/Shareddrives/HLT/dataset/gold/umberto_preprocessing/merged_umberto_preprocessing_test.csv', index=False)

In [ ]:
merged_test_proc = pd.read_csv('/content/drive/Shareddrives/HLT/dataset/gold/umberto_preprocessing/merged_umberto_preprocessing_test.csv')

In [ ]:
merged_test_proc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3703 entries, 0 to 3702
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   anonymized_tweet_id     3703 non-null   int64 
 1   anonymized_text         3703 non-null   object
 2   label                   3703 non-null   int64 
 3   dataset                 3703 non-null   object
 4   retweet_count           3703 non-null   int64 
 5   favorite_count          3703 non-null   int64 
 6   anonymized_user_id      3703 non-null   int64 
 7   statuses_count          3703 non-null   int64 
 8   followers_count         3703 non-null   int64 
 9   friends_count           3703 non-null   int64 
 10  anonymized_description  3703 non-null   object
dtypes: int64(8), object(3)
memory usage: 318.4+ KB


In [ ]:
merged_test_proc.head()

,anonymized_tweet_id,anonymized_text,label,dataset,retweet_count,favorite_count,anonymized_user_id,statuses_count,followers_count,friends_count,anonymized_description
0,424801448454884,questo anno [hashtag] babbo natale [/hashtag] ...,1,test_politics,486,1827,587049380396341,25758,132172,13522,rigore è quando arbitro fischia non vero bosko...
1,775194088981616,adesso che in mezzo alla strada grazie al verg...,1,test_politics,581,1996,924711002016921,27794,39224,4682,agisci in modo da trattare l umanità in te com...
2,386928936347190,circondatevi di persone che non sono diventate...,0,test_politics,47,252,62043137600220,18643,10758,8690,[url]
3,957546674360002,seriamente per capire se un ladro mi entra in ...,0,test_politics,7,30,801758473146114,132391,5621,2490,suora dal [number] [user] bloccami !
4,530526299555950,che poi è probabile che [hashtag] spataro [/ha...,0,test_politics,0,0,334768891377767,31009,423,306,icastico interista seguo la palla al cesto so ...
